### Задача 1: применяем PCA-трансформацию
Модифицируйте файл train.py - добавьте в пайплайн обучения модели сжатие размерности до n_components=2 с помощью PCA и обучите модель в докере на "сжатых" данных. Сохраните полученный объект pca_transformer.pkl, который умеет выполнять сжатие данных.

Решением домашки считается модифицированный файл train.py

In [1]:
import pickle
import os
import logging
import numpy as np
from pathlib import Path
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
# загрузка данных



class TransformerPCA(object):
    def __init__(self, source: Path):
        self.data_source = np.genfromtxt(source, delimiter=',', skip_header=1)
        self.X = self.data_source[:, :3]
        self.y = self.data_source[:, 3]
    

    def PCA_transform(self):
        pca_transformer = PCA(n_components=2).fit(self.X)
        x_transformed = pca_transformer.transform(self.X)
        return x_transformed

    
    def learn(self):
        # обучение модели
        clf = DecisionTreeClassifier(max_depth=3, random_state=42)
        clf.fit(self.X, self.y)
        # сохраняем модель внутри контейнера в директории /www/classifier
        path_to_save_cf = 'clf.pkl'
        with open(path_to_save_cf, 'wb') as f:
            pickle.dump(clf, f)
            logging.info('Модель обучена и сохранена в %s' % Path().absolute())
        # обучение модели на сжатых данных
        clf.fit(self.PCA_transform(), self.y)
        # Сохраняем модель обученную на сжатых данных
        path_to_save_tf = './data/pca_transformer.pkl'
        with open(path_to_save_tf, 'wb') as f:
            logging.info(f'psna_transformer  сохранен в файле {path_to_save_tf}')
            pickle.dump(clf, f)
    
    
    def pipeline(self):
        self.learn()
   




path_to_data = 'data/client_segmentation.csv'
pca_tf = TransformerPCA(path_to_data)
pca_tf.pipeline()

2021-05-12 22:44:36,468 | INFO     | <ipython-input-1-427f6ae0:36   | Модель обучена и сохранена в C:\Users\Константин\Desktop\Алена\jupyter
2021-05-12 22:44:36,491 | INFO     | <ipython-input-1-427f6ae0:42   | psna_transformer  сохранен в файле ./data/pca_transformer.pkl


### Домашнее задание: трансформация входных фичей на лету
Модифицируйте файл service.py: добавьте загрузку объекта для трансформации tsne_tansformer.pkl и применяйте её в докере для трансформации набора входных фич в сжатые:

[x1, x2, x3] -> [x1_tsne, x2_tsne]
Соответственно, predict надо выполнять на сжатых фичах

Решением домашки считается модифицированный файл service.py

In [ ]:
"""
Умеет выполнять классификацию клиентов по трём фичам
Запускаем из python3:
    python3 service.py
Проверяем работоспособность:
    curl http://127.0.0.1:5000/
"""
import json
import http.server
import logging
import os
import pickle
import socketserver
import sys
from http import HTTPStatus
from re import compile

import numpy as np
from sklearn.tree import DecisionTreeClassifier

# файл, куда посыпятся логи модели

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(filename="/www/classifier/data/service.log", level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """
        Выдираем параметры из GET-запроса
    """
    params_list = params.split('&')
    params_dict = {'x1': None, 'x2': None, 'x3': None}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    return params_dict


class Handler(http.server.SimpleHTTPRequestHandler):
    """Простой http-сервер"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_response(self) -> dict:
        """Пример запроса
        
        http://0.0.0.0:5000/classifier/?x1=1&x2=-2.2&x3=1.05
        """
        response = {'ping': 'ok'}
        params_parsed = self.path.split('?')
        if len(params_parsed) == 2 and self.path.startswith('/classifier'):
            params = params_parsed[1]
            params_dict = parse_params(params)
            response = params_dict
            user_features = np.array([params_dict['x1'], params_dict['x2'], params_dict['x3']]).reshape(1, -1)
            predicted_class = int(classifier_model.predict(user_features)[0])
            logging.info('predicted_class %s' % predicted_class)
            response.update({'predicted_class': predicted_class})
        elif self.path.startswith('/ping/'):
            response = {'message': 'pong'}

        return response

    def do_GET(self):
        # заголовки ответа
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(self.get_response()).encode())


logging.info('Загружаем обученные модели')
with open('./data/pca_transformer.pkl', 'rb') as f:
    transformer_model = pickle.load(f)
    logging.info('Модель, обученная на сжатых данных загружена')

path_to_save_cf = 'clf.pkl'
with open(path_to_save_cf, 'rb') as f:
    classifier_model = pickle.load(f)   
    logging.info('Модель загружена')

if __name__ == '__main__':
    classifier_service = socketserver.TCPServer(('', 5001), Handler)
    classifier_service.serve_forever()

2021-05-12 22:44:36,889 | INFO     | <ipython-input-2-bbd88b5c:73   | Загружаем обученные модели
2021-05-12 22:44:36,897 | INFO     | <ipython-input-2-bbd88b5c:76   | Модель, обученная на сжатых данных загружена
2021-05-12 22:44:36,897 | INFO     | <ipython-input-2-bbd88b5c:81   | Модель загружена


### Домашнее задание: Используем Flask
Перепишите сервис на использование Flask. Вы можете взять готовый базовый образ с Flask, либо добавить установку в тот контейнер, который есть - это нужно сделать в Dockerfile

In [ ]:
import pickle
import logging
from flask import Flask, request, make_response

app = Flask(__name__)
LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(filename="/www/classifier/data/service.log", level=logging.INFO, format=LOG_FORMAT)

@app.route('/ping')
def ping():
    return {'message': 'pong'}

@app.route('/predict')
def predict():
    args = request.args
    try:
        x1 = float(args['x1'])
        x2 = float(args['x2'])
        x3 = float(args['x3'])
    except:
        return {'msg': 'Wrong data'}
    input_feautures = tf.transform([[x1, x2, x3]])
    prediction = clf.predict(input_feautures)[0]
    logging.info('predicted_class %s' % predicted_class)
    response = make_response({'predicted_class': predicted_class})
    return response

with open('./data/pca_transformer.pkl', 'rb') as f:
    tf = pickle.load(f)
    logging.info('Модель, обученная на сжатых данных загружена')

path_to_save_cf = 'clf.pkl'
with open(path_to_save_cf, 'rb') as f:
    clf = pickle.load(f)   
    logging.info('Модель загружена')
    
#app.run()

### Домашнее задание: строим KNN
В реальной жизни KNN-рекомендатель не стоит делать на основе sklearn.neighbors.NearestNeighbors - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является пакет implictit. В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки

Почитайте документацию по модулю implicit.nearest_neighbours.CosineRecommender. Обучите KNN-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [ ]:
import pandas as pd
import numpy as np

content_views = pd.read_csv(
    'recsys_data/content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)


print('Количество просмотров %s' % content_views.user_id.count())

content_views.head(3)

In [ ]:
content_description = pd.read_csv(
    'recsys_data/content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)

print('Количество доступного контента %s' % content_description.content_id.count())

content_description.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# ереиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# нова переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)


content_views.head()

In [ ]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.user_index.values, content_views.item_index.values)),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

In [ ]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    "Размер обучающей выборки %d пользователей, размер валидационной выборки %d пользователей"
    % (train_ids.size, test_ids.size)
)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# обучаемся только на тренировочной части пользователей
model_knn.fit(user_item[train_ids,:])

In [ ]:
class ColaborativeFilteringKNNRecommender:
    def __init__(self, knn_model, user_item_matrix, num_neighbors):
        self.knn_model = knn_model
        self.user_item_matrix = user_item_matrix
        self.num_neighbors = num_neighbors
        self.top_recs = 50
    
    def make_recs(self, user_history: csr_matrix, top_recs: int):
        neighbors = model_knn.kneighbors(
            random_user_history,
            self.num_neighbors,
            return_distance=False
        )[0]
        full_recs = user_item[neighbors,:].max(axis=0)
        # рекомендации - это то, что насмотрели ближайшие соседи
        user_history_ids = user_history.nonzero()[1]
        # последовательность id того контента, который смотрели ближайшие соседи
        full_recs_ids = full_recs.nonzero()[1][:self.top_recs]
        # исключаем из рекомендаций то, что уже было у упользователя в историии
        success_recs = np.array([i for i in full_recs_ids if i in user_history_ids])
        print("Число успешных рекомендаций %d из %d" % (success_recs.size, top_recs))
        
        return np.array([i for i in full_recs_ids if i not in user_history_ids])[:10]


# объект рекомендателя
recommender = ColaborativeFilteringKNNRecommender(
    knn_model=model_knn,
    user_item_matrix=user_item,
    num_neighbors=10
)

In [ ]:
# пример рекомендаций для случайного пользователя
random_user_index = 934 #np.random.choice(test_ids)
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = recommender.make_recs(random_user_history, top_recs=10)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

In [ ]:
# -- ВАШ КОД ТУТ --
from implicit.nearest_neighbours import CosineRecommender
# Настраиваем рекомендатель
cr = CosineRecommender(K=50, num_threads=0)
# Обучаем
cr.fit(user_item[train_ids,  :])
# Получаем рекоменлацию


In [ ]:
cr.recommend(random_user_index, user_item[train_ids, :])

### Домашнее задание: Item to Item
Решите задачу c2c рекомендаций - вызовите метод similar_items для item_id=1

In [ ]:
cr.similar_items(1)

### Домашнее задание: обучаем Implicit
Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [ ]:
from implicit.als import AlternatingLeastSquares

recommender = AlternatingLeastSquares()
recommender.fit(user_item[train_ids,:])

recommender.recommend(random_user_index, user_item[train_ids,:])

### Домашнее задание на метрики
Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

precision
recall
precision@5

In [21]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

recall = len(set(user_interactions)&set(user_recs)) / len(set(user_interactions))
precision = len(set(user_interactions)&set(user_recs)) / len(set(user_recs))
print(f'recall = {recall}')
print(f'precision = {precision}')
# Вычисляем precision@5
k = 5
ap, i = 0, 0
# Общий смысл оцениваем рекомендации по их месту в истории пользователя
while i < len(user_recs):
    if user_recs[i] in user_interactions:
        index = user_interactions.index(user_recs[i])+1
        ap += 1/index
        k -= 1
    i += 1
        
print(f'precision@5 = {ap*(1/5)}')     
        

recall = 0.1
precision = 0.5
precision@5 = 0.05183403900280742
